In [1]:
import glob
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import os
import time
from urllib.parse import urlparse

all_txt_files = glob.glob(os.getcwd()+'/prod_urls/*.txt')
results_dir = os.getcwd()+'/results'

In [2]:
all_txt_files

['/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/Self-Help Books.txt',
 '/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/History and Archaeology Books.txt',
 '/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/Biographies, Memoirs and General Non-Ficton Books.txt',
 '/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/Philosophy and Religion Books.txt',
 '/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/Healthy Living and Wellness Books.txt',
 '/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/Arts, Language and Linguistic Books.txt',
 '/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/Literature Books.txt',
 '/home/bandham/projects-data-science/webscraping-product-details-flipkart/prod_urls/Fiction Books.txt']

In [3]:
genres_and_links = dict()

for filename in all_txt_files:
    with open(filename,'r') as file:
        relative_filename = filename.split('/')[-1]
        genres_and_links[relative_filename.replace('.txt','')] = file.readlines()

In [4]:
for kk,vv in genres_and_links.items():
    print(kk,len(vv))

Self-Help Books 2000
History and Archaeology Books 2000
Biographies, Memoirs and General Non-Ficton Books 1000
Philosophy and Religion Books 1000
Healthy Living and Wellness Books 1000
Arts, Language and Linguistic Books 1000
Literature Books 2000
Fiction Books 2000


In [5]:
dump_urls = ['https://www.thepythoncode.com/article/download-web-page-images-python','https://www.geeksforgeeks.org/create-a-directory-in-python/','https://pynative.com/','http://facebook.com/','https://www.amazon.in/','https://www.tutorialspoint.com/','https://docs.python.org/3.4/library/re.html']
import random

In [6]:
ultimate_dataframes = dict()

In [7]:
driver = None

for genre, links in genres_and_links.items():
    
    results_dict = dict()

    list_of_book_titles = list()
    list_of_prices = list()
    list_of_publishers = list()
    list_of_pages = list()
    list_of_isbns = list()
    list_of_genres = list()
    list_of_ratings = list()
    list_of_authors = list()
    list_of_book_summaries = list()
    list_of_image_names = list()

    main_counter = 0
    for link in links:
        
        requests.get(dump_urls[random.randint(0,6)])
        time.sleep(1)
        resp = requests.get(link)
        
        soup = BeautifulSoup(resp.content,'html.parser')
        
        book_title_ele = soup.find('span',{'class':'_35KyD6'})
        book_title = book_title_ele.get_text()
        
        price = soup.find('div',{'class':'_1vC4OE _3qQ9m1'}).get_text()
        
        spec_tbody_elements = soup.find_all('tr',{'class':'_3_6Uyw row'})
        publisher = ''
        for tbody in spec_tbody_elements:
            tr_elements = tbody.find_all('td')
            if len(tr_elements)==2:
                if tr_elements[0] and tr_elements[0].text=='Imprint':
                    publisher = tr_elements[1].text
            else:
                publisher = 'NA'
    
        no_of_pages = 'NA'
        isbn = 'NA'
        highlights_li_elements = soup.find_all('li',{'class':'_2-riNZ'})
        for li in highlights_li_elements:
            if li and li.get_text() and li.get_text().startswith('Pages'):
                no_of_pages = li.get_text().split(': ')[1]
            if li and li.get_text() and li.get_text().startswith('ISBN'):
                isbn = li.get_text().split(': ')[1]
        
        rating = 'NA'
        rating_parent_element = soup.find('div',{'class':'_3ors59'})
        if rating_parent_element:
            rating_element = rating_parent_element.find('div',{'class':'hGSR34'})
            if rating_element:
                rating = rating_element.get_text()
        
        author = 'NA'
        book_summary = 'NA'
        book_desc_list = list()
        author_description_elements = soup.find_all('div',{'class':'_1y9a40'})
        for element in author_description_elements:
            if element.find('div',{'class':'_1oCqc9'}) and element.find('div',{'class':'_1oCqc9'}).get_text()=='Author':
                author = element.find('div',{'class':'_3cpW1u'}).get_text().strip()
            if element.find('div',{'class':'_1oCqc9'}) and element.find('div',{'class':'_1oCqc9'}).get_text()=='Description':
                if element.find('div',{'class':'_3cpW1u'}):
                    book_summary = re.sub('\\s+',' ',element.find('div',{'class':'_3cpW1u'}).get_text())
                #desc_p_elements = element.find_all('p');
                #book_desc_list = [re.sub('\\s+',' ',p.get_text()) for p in desc_p_elements if p]
                #book_summary = ''.join(book_desc_list)
                
        #print(help(soup.find('div',{'class':'_2rDnao'})))
        #print(help(soup.find('div',{'class':'_2rDnao'}).get_attribute_list))
        
        image_urls = set()
        image_elements = soup.find_all('div',{'class':'_2_AcLJ'})
        image_url = None
        for ele in image_elements:
            if ele.has_attr('style'):
                if '(' in ele.get('style') and ')' in ele.get('style'):
                    image_url = ele.get('style').split('(')[1].split(')')[0]
                if image_url is not None:
                    image_url = re.sub('\\w+?q=\\d+','',image_url)
                    image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
                    url_parse_results = urlparse(image_url)
                    if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                        image_urls.add(image_url)
                
        if len(image_urls)==0:
            if driver is None:
                driver = webdriver.Firefox(executable_path="./geckodriver")
            driver.get(dump_urls[random.randint(0,6)])
            time.sleep(1)
            driver.get(link)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source,'html.parser')
            image_divs = soup.find_all('div',{'class':'_1ov7-N'})
            for image_div in image_divs:
                img_elements = image_div.find_all('img')
                for img in img_elements:
                    if img.has_attr('src'):
                        image_url = img.get('src')
            if image_url is not None:
                image_url = re.sub('\\w+?q=\\d+','',image_url)
                image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
                url_parse_results = urlparse(image_url)
                if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                    image_urls.add(image_url)
                
        i_counter = 1
        i_names = list()
        for i_url in image_urls:
            genre_images_dir = results_dir + '/images/' + genre
            if not os.path.exists(genre_images_dir):
                os.mkdir(genre_images_dir)
            i_name = genre_images_dir + '/' +str(main_counter+1)+'.'+str(i_counter)+'.jpeg'
            
            requests.get(dump_urls[random.randint(0,6)])
            time.sleep(1)
            i_response = requests.get(i_url)
                
            with open(i_name, 'wb') as f:
                f.write(i_response.content)
                
            i_names.append(i_name)
            #print(i_name)
            i_counter +=1
        

        #print(book_title)#'title_bindtype_language-'
        #print(price)
        #print(publisher)
        #print(no_of_pages)
        #print(isbn)
        #print(genre)
        #print(rating)
        #print(author)
        #print(book_summary)
        #print(image_urls)
        
        list_of_book_titles.append(book_title)
        list_of_prices.append(price)
        list_of_publishers.append(publisher)
        list_of_pages.append(no_of_pages)
        list_of_isbns.append(isbn)
        list_of_genres.append(genre)
        list_of_ratings.append(rating)
        list_of_authors.append(author)
        list_of_book_summaries.append(book_summary)
        list_of_image_names.append(', '.join(i_names))
        
        #print('-----------------')
        
        main_counter += 1
        if main_counter==100:
            break;
    
    results_dict['book_title'] = list_of_book_titles
    results_dict['price'] = list_of_prices
    results_dict['publisher'] = list_of_publishers
    results_dict['no_of_pages'] = list_of_pages
    results_dict['isbn'] = list_of_isbns
    results_dict['genre'] = list_of_genres
    results_dict['rating'] = list_of_ratings
    results_dict['author'] = list_of_authors
    results_dict['book_summary'] = list_of_book_summaries
    results_dict['images'] = list_of_image_names
    
    df = pd.DataFrame(results_dict)
    export_file_name = results_dir + '/' + genre + '.csv'
    df.to_csv(export_file_name,index=False)
    ultimate_dataframes[genre] = df

In [8]:
if driver is not None:
    driver.quit()

In [9]:
list(ultimate_dataframes.values())[0].head()

,book_title,price,publisher,no_of_pages,isbn,genre,rating,author,book_summary,images
0,"You Can Win (English, Paperback, Khera Shiv)",₹245,Bloomsbury India,308,"9789382951711, 9789382951711",Self-Help Books,4.5,Khera Shiv,You Can Win: A Step By Step Tool For Top Achi...,/home/bandham/projects-data-science/webscrapin...
1,"Sochiye Aur Ameer Baniye (Hindi, Paperback, H...",₹139,Manjul Publishing House Pvt Ltd,275,"9788186775554, 9788186775554",Self-Help Books,4.3,Hill Napoleon,The book discusses the power of positive and ...,/home/bandham/projects-data-science/webscrapin...
2,"The 5 AM Club (Kannada, Paperback, unknown)",₹225,Jaico Publishing House,336,"9789388423939, 9789388423939",Self-Help Books,3.2,unknown,NA,/home/bandham/projects-data-science/webscrapin...
3,How to Win Friends and Influence People - The ...,₹199,Qford Educare Family,236,"9789380494340, 9789380494340",Self-Help Books,4.4,Carnegie Dale,How to Win Friends & Influence People is a cl...,/home/bandham/projects-data-science/webscrapin...
4,"Believe in Yourself (English, Paperback, Murp...",₹131,General Press,110,"9789388118385, 9789388118385",Self-Help Books,4.3,Murphy Joseph,NA,/home/bandham/projects-data-science/webscrapin...


In [15]:
df1 = list(ultimate_dataframes.values())[0]
df1[df1['book_summary']=='NA']

,book_title,price,publisher,no_of_pages,isbn,genre,rating,author,book_summary,images
2,"The 5 AM Club (Kannada, Paperback, unknown)",₹225,Jaico Publishing House,336,"9789388423939, 9789388423939",Self-Help Books,3.2,unknown,NA,/home/bandham/projects-data-science/webscrapin...
4,"Believe in Yourself (English, Paperback, Murp...",₹131,General Press,110,"9789388118385, 9789388118385",Self-Help Books,4.3,Murphy Joseph,NA,/home/bandham/projects-data-science/webscrapin...
5,"Jeetna Hai to Jid Karo (Hindi, Paperback, Hun...",₹108,Arihant Publishers,182,"9789351767053, 9789351767053",Self-Help Books,4.3,Hundiwala S.,NA,/home/bandham/projects-data-science/webscrapin...
11,"Socho Aur Amir Ho Jao (Hindi, Paperback, Hill...",₹125,Navyug Publishers,264,"9788175993563, 9788175993563",Self-Help Books,4.3,Hill Napolean,NA,/home/bandham/projects-data-science/webscrapin...
12,"Chanakya Neeti (Bengali, Paperback, Mishra Ra...",₹114,Diamond Pocket Books,152,"9788128834127, 9788128834127",Self-Help Books,4.4,Mishra Rajeshwar,NA,/home/bandham/projects-data-science/webscrapin...
14,"The Subtle Art of Not Giving a F*ck (English,...",₹225,HarperCollins,NA,"9781441711441, 9781441711441",Self-Help Books,4.3,Manson Mark,NA,/home/bandham/projects-data-science/webscrapin...
20,How to Stop Worrying and Start Living (Englis...,₹196,General Press,288,"9789380914817, 9789380914817",Self-Help Books,4.5,Carnegie Dale,NA,/home/bandham/projects-data-science/webscrapin...
23,"The Power of Your Subconscious Mind (English,...",₹261,Dover Publications Inc.,208,"9780486478999, 9780486478999",Self-Help Books,4.3,Murphy Joseph,NA,/home/bandham/projects-data-science/webscrapin...
28,Little Black Book for Stunning Success + Tools...,₹262,Jaico Publishing House,288,"9788184959895, 9788184959895",Self-Help Books,4.4,Sharma Robin,NA,/home/bandham/projects-data-science/webscrapin...
31,"Million Dollar Habits (English, Paperback, Tr...",₹268,Entrepreneur Press,272,"9781599186528, 9781599186528",Self-Help Books,4.7,Tracy Brian,NA,/home/bandham/projects-data-science/webscrapin...
